# Python More Examples

In [1]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In [2]:
from addutils import css_notebook
css_notebook()

## 1 Working with Win32Com - Excel example

`win32com.client` allows Python to control most windows applications. In this case we automate an **Excel** application.
NB: the following examples assume that Microsoft Excel is installed on the current PC.

In [3]:
from win32com.client import Dispatch   # Import the 'win32com' module
import os

ImportError: No module named 'win32com'

In [ ]:
xlApp = Dispatch('Excel.Application')  # reference to an excel instance

Now Excel has been started but is invisible. Let's make Excel visible. Then we create a new Sheet named *'My First Try'*:

In [ ]:
xlApp.Visible = 1                      # Make Excel visible
xlBook = xlApp.Workbooks.Add()         # New workbook (standard three sheets)
xlSheet = xlBook.Sheets(1)             # Handle to the first sheet (ONE BASED!)
xlSheet.Name = 'My First Try'          # Change sheet name

Cells can be initialized (here we assign two text values) and formatted from code:

In [ ]:
rng = xlSheet.Range("C3","D3")
rng.Value = ('Hello', 'World!')
rng.NumberFormat = '@'  # Assign the 'Text' cell format
rng.Font.Bold = True
rng.Font.Name = "Arial"
rng.Font.Size = 12
rng.Interior.ColorIndex = 8
rng.Interior.Pattern = 2

In [ ]:
v = xlSheet.Cells(3,4).Value           # Read a single value from a cell
print(v, type(v))

How to assign numerical values and formulas to cells (notice that formulas are assigned as strings):

In [ ]:
xlSheet.Cells(4,4).Value == None       # Check if the cell is empty
xlSheet.Cells(5,4).Value = '12'
xlSheet.Cells(6,4).Value = '15'
xlSheet.Cells(7,4).Value = '=D5+D6'    # Formulas

We can write and read values to whole cell ranges as well:

In [ ]:
rng = xlSheet.Range(xlSheet.Cells(3,3), xlSheet.Cells(7,4))
rng.Value

In [ ]:
vals=tuple([(x,y,z) for x,y,z in zip('abcdefghijklmn',xrange(14),xrange(14))])
xlSheet.Range("C8","E21").Value = vals

Save the workbook with a given name:

In [ ]:
current_dir = os.path.abspath(os.path.curdir)
file_path = os.path.join(current_dir, 'tmp', 'my_excel_file.xlsx')
print(file_path)
xlBook.SaveAs(file_path)

To quit the application do this:

In [ ]:
xlBook.Close(SaveChanges=0)           # Book force closing without saving
xlApp.Quit()                          # Quit the excel App
del xlApp                             # Release the local reference

## 2 Sending an e-mail from a gmail account

This example is about sending mails using Python smtplib through Gmail SMTP.

In [4]:
sender = 'MYNAME@gmail.com'
password = 'MYPASSWORD'
recipient = 'RECIPIENT-EMAIL'
subject = 'A G-mail using Python'
body = 'This is the body of the message'

The smtplib module defines an SMTP client session object that can be used to send mail to any Internet machine with an SMTP or ESMTP listener daemon.
The first step is to create a SMTP connection to the server. The `smtplib.SMTP` class encapsulates an SMTP connection. If the optional host and port parameters are given, the SMTP `connect()` method is called with those parameters during initialization. An `SMTPConnectError` is raised if the specified host doesn’t respond correctly. The optional timeout parameter specifies a timeout in seconds for blocking operations like the connection attempt (if not specified, the global default timeout setting will be used).

Next we will identify ourself to an ESMTP server using HELO. The SMTP HELO clause is the stage of the SMTP protocol where SMTP servers get introduced each other. EHLO is just like HELO except that the server’s response text provides computer-readable information about the server’s abilities.

Remember Google’s SMTP server is ‘smtp.gmail.com’ and the port is 587.

In [5]:
import smtplib
session = smtplib.SMTP('smtp.gmail.com', 587)

Next we call `SMTP.starttls` function to put the SMTP connection in TLS (Transport Layer Security) mode. All SMTP commands that follow will be encrypted. You should then call `ehlo()` again. If keyfile and certfile are provided, these are passed to the socket module’s `ssl()` function.If there has been no previous EHLO or HELO command this session, this method tries ESMTP EHLO first.

In [6]:
session.starttls()
session.ehlo()

(250,
 b'smtp.gmail.com at your service, [93.50.182.77]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN XOAUTH OAUTHBEARER\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

OK, now we are safe to login to the server using SMTP.login(user, password). After successful login we use SMTP.sendmail(from_addr, to_addrs, msg[, mail_options, rcpt_options]) to send mails via Gmail.

In [7]:
session.login(sender, password)

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/answer/14257 ka10sm18528678wjc.30 - gsmtp')

In [ ]:
headers = ["from: " + sender,
           "subject: " + subject,
           "to: " + recipient,
           "mime-version: 1.0",
           "content-type: text/html"]
headers = "\r\n".join(headers)

In [ ]:
session.sendmail(sender, recipient, headers + "\r\n\r\n" + body)

## 3 Automating google search

Using google webservices API is possible to query google search engine. Say we are going to search "barack obama"

In [11]:
import json
import urllib

searchfor = 'barack obama'

To do this we need to create a specially crafted url to pass to the google api. `urllib` is a standard `python` library that allows to create and retrive URL(s).


In [13]:
query = urllib.parse.urlencode({'q': searchfor})
url = 'http://ajax.googleapis.com/ajax/services/search/web?v=1.0&{}'.format(query)
search_response = urllib.request.urlopen(url)
search_results = search_response.read()

Now we have a json encoded string containing our results, we are going to use `json.loads` in order to get what we are interested to (say Total results and top hits).

In [28]:
results = json.loads(search_results.decode(encoding='UTF-8'))
data = results['responseData']
print('Total results: {}'.format(data['cursor']['estimatedResultCount']))
hits = data['results']
print('Top {} hits:'.format(len(hits)))
for h in hits:
    print(' ', h['url'])
print('For more results, see {}'.format(data['cursor']['moreResultsUrl']))

<class 'str'>
Total results: 13400000
Top 4 hits:
  https://www.barackobama.com/
  https://en.wikipedia.org/wiki/Barack_Obama
  https://www.whitehouse.gov/administration/president-obama
  https://twitter.com/barackobama
For more results, see http://www.google.com/search?oe=utf8&ie=utf8&source=uds&start=0&hl=en&q=barack+obama


---

Visit [www.add-for.com](<http://www.add-for.com/IT>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.